# 一、描述符示例：属性验证

In [1]:
class Quantity:
    def __init__(self, storage_name) -> None:
        self.storage_name = storage_name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self.storage_name]

In [2]:
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')

    def __init__(self, description, weight, price) -> None:
        self.description = description
        self.weight = weight  # 这里之前特性已经激活
        self.price = price

    def subtotal(self):
        return self.weight * self.price


In [4]:
# truffle = LineItem('White truffle', 100, 0)

为储存属性自动命名

In [5]:
class Quantity:

    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
            if value > 0:
                instance.__dict__[self.storage_name] = value
            else:
                msg = f'{self.storage_name} must be > 0'
                raise ValueError(msg)


In [6]:
class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price) -> None:
        self.description = description
        self.weight = weight  # 这里之前特性已经激活
        self.price = price

    def subtotal(self):
        return self.weight * self.price



一种新型描述符：描述符的继承重用

In [7]:
import abc

class Validated(abc.ABC):

    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        value = self.validate(self.storage_name, value)
        instance.__dict__[self.storage_name] = value

    @abc.abstractmethod
    def validate(self, name, value):
        """返回通过验证的值，或者抛出ValueError"""

In [9]:
class Quantity(Validated):
    """数值大于零"""
    def validate(self, name, value):
        if value <= 0:
            raise ValueError(f'{name} must be > 0')
        return value

class NonBlank(Validated):

    def validate(self, name, value):
        value = value.strip()
        if not value:
            raise ValueError(f'{name} cannot be blank')
        return value

In [10]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price) -> None:
        self.description = description
        self.weight = weight  # 这里之前特性已经激活
        self.price = price

    def subtotal(self):
        return self.weight * self.price


In [65]:
help(LineItem.weight)

Help on Quantity in module __main__:




# 二、覆盖型描述符与非覆盖型描述符对比

In [11]:
### 辅助函数，仅用于显示 ###
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return f'<{cls_name(obj)} object>'

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print(f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')

In [12]:
class Overriding:
    """数据描述符或强制描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [14]:
class OverridingNoGet:
    """没有__get__方法的覆盖型描述符"""
    def __set__(self, instance, value):
        print_args('set', self, instance, value)


In [15]:
class NonOverriding:
    """非数据描述符或遮盖型描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)


In [16]:
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):
        print(f'-> Managed.spam({display(self)})')

In [17]:
obj = Managed()

In [18]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [20]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [21]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [23]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [24]:
obj.__dict__['over'] = 7

In [26]:
vars(obj)

{'over': 7}

In [25]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [27]:
obj.over_no_get

In [28]:
Managed.over_no_get

In [29]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [30]:
obj.over_no_get

In [31]:
obj.__dict__['over_no_get'] = 7

In [32]:
obj.over_no_get

7

In [33]:
obj.over_no_get = 9

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 9)


In [34]:
obj.over_no_get

7

In [35]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [36]:
obj.non_over = 7

In [37]:
obj.non_over

7

In [38]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [39]:
del obj.non_over

In [40]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [41]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [42]:
Managed.over = 1

In [44]:
Managed.non_over = 2

In [45]:
Managed.over_no_get = 3

In [43]:
Managed.over

1

In [46]:
obj.over, obj.non_over, obj.over_no_get

(7, 2, 7)

In [47]:
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x0000020ABA4A13A0>>

In [48]:
Managed.spam

<function __main__.Managed.spam(self)>

In [49]:
obj.spam = 7

In [50]:
obj.spam

7

In [51]:
import collections

class Text(collections.UserString):
    def __repr__(self) -> str:
        return f'Text({self.data!r})'
    def reverse(self):
        return self[::-1]

In [52]:
word = Text('forward')
word

Text('forward')

In [53]:
word.reverse()

Text('drawrof')

In [54]:
Text.reverse(Text('backword'))

Text('drowkcab')

In [55]:
type(Text.reverse), type(word.reverse)

(function, method)

In [56]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

In [59]:
Text.reverse.__get__(None, Text)

<function __main__.Text.reverse(self)>

In [60]:
word.reverse

<bound method Text.reverse of Text('forward')>

In [64]:
word.reverse.__self__ is word

True

In [63]:
word.reverse.__func__ is Text.reverse

True